In [1]:
# import json
import numpy as np
import requests
from io import BytesIO
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from skimage.filters import threshold_otsu
from skimage.io import imread, imsave
from skimage.color import rgb2gray
from skimage.measure import label, regionprops
import json
import os
import uuid
from PIL import Image
import random
import math
import imageio
import imgaug as ia
from imgaug import augmenters as iaa
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage
import glob

In [2]:
def reset(percent=20):
    return random.randrange(100) < percent

In [3]:
# Training and Validate Sets
# degs = [45,90,135,180,225,270,315]
degs = [90,180,270,360,450,540,630]
img_bbs = []
count = 1
phase_count = {
    "train": 1,
    "test": 1
}

labels = os.path.join('training.json')
with open(labels,'r') as json_file:
    datas = json.load(json_file)
    for data in datas:
        read_path = (data['image']['pathname'])[1:]
        print(read_path)
        phase = "train" if (count % 2) == 0 else "test"            
        img_name_new = "pv-" + phase + "-{:010d}".format(phase_count[phase])
        print(img_name_new)
        save_path = os.path.join('../data/custom/images',img_name_new+".jpg")
        img = Image.open(read_path)
        objects = data['objects']
        rects = []
        result = ""
        img = np.array(img)
        h,w,c = img.shape
        RBCs_flag = True
        for obj in objects:
            minr = obj['bounding_box']['minimum']['r']
            minc = obj['bounding_box']['minimum']['c']
            maxr = obj['bounding_box']['maximum']['r']
            maxc = obj['bounding_box']['maximum']['c']
            x1 = minc / w
            y1 = minr / h
            bw = (maxc - minc) / w
            bh = (maxr - minr) / h
            xc = x1 + (bw / 2)
            yc = y1 + (bh / 2)
            category = obj['category']
            if (category == "red blood cell") and (RBCs_flag or phase == "test"):
                result += "\n"+ "0 " + str(xc) + " " + str(yc) + " " + str(bw) + " " + str(bh)
            elif category == "ring" or category == "trophozoite" or category == "schizont" or category == "gametocyte":
                result += "\n" + "1 " + str(xc) + " " + str(yc) + " " + str(bw) + " " + str(bh)
                rects.append(BoundingBox(x1=minc,y1=minr,x2=maxc,y2=maxr,label=1))
        result = result[1:]
        if len(result) == 0:
            count += 1
            phase_count[phase] += 1
            continue
        Image.fromarray(img).save(save_path)
        f = open(os.path.join('../data/custom/labels',img_name_new+".txt"),'w')
        f.write(result)
        f.close()
        bbs = BoundingBoxesOnImage(rects,shape=img.shape)
        count += 1
        phase_count[phase] += 1
        if phase == "train":
            for deg in degs:
                image_aug, bbs_aug = iaa.Affine(rotate=deg,translate_percent={"x": (-0.25, 0.25), "y": (-0.25, 0.25)},mode="reflect")(image=img, bounding_boxes=bbs)
                clip_off_bbs = bbs_aug.remove_out_of_image(fully=True,partly=True).clip_out_of_image()
                h,w,c = image_aug.shape
                result = ""
                img_name_new = "pv-" + phase + "-{:010d}".format(phase_count[phase])
                for bb in clip_off_bbs.bounding_boxes:
                    minx = int(bb.x1)
                    miny = int(bb.y1)
                    maxx = int(bb.x2)
                    maxy = int(bb.y2)
                    bb_w = (maxx-minx)/w
                    bb_h = (maxy-miny)/h
                    bb_xc = (minx + ((maxx-minx)/2))/w
                    bb_yc = (miny + ((maxy-miny)/2))/h
                    result += "\n" + str(bb.label)+" "+str(bb_xc)+" "+str(bb_yc)+ " "+str(bb_w)+ " "+str(bb_h)
                if len(clip_off_bbs.bounding_boxes) > 0:
                    print("{} {}".format(phase_count[phase],deg))
                    print(img_name_new)
                    result = result[1:]
                    save_path = os.path.join('../data/custom/images',img_name_new+".jpg")
                    Image.fromarray(image_aug).save(save_path)
                    f = open(os.path.join('../data/custom/labels',img_name_new+".txt"),'w')
                    f.write(result)
                    f.close()
                    phase_count[phase] += 1

images/8d02117d-6c71-4e47-b50a-6cc8d5eb1d55.png
pv-test-0000000001
images/10be6380-cbbb-4886-8b9e-ff56b1710576.png
pv-train-0000000001
images/6b14c855-8561-417c-97a4-63fa552842fd.png
pv-test-0000000002
images/13099edb-35d9-438f-b093-2cf2ebf9d255.png
pv-train-0000000002
3 90
pv-train-0000000003
4 180
pv-train-0000000004
5 360
pv-train-0000000005
6 540
pv-train-0000000006
7 630
pv-train-0000000007
images/2559636b-f01a-4414-93da-210c3b12d153.png
pv-test-0000000003
images/0dcca702-a4ef-4fb3-a940-9c0c140b21c7.png
pv-train-0000000008
9 90
pv-train-0000000009
10 180
pv-train-0000000010
11 270
pv-train-0000000011
12 360
pv-train-0000000012
13 450
pv-train-0000000013
14 540
pv-train-0000000014
15 630
pv-train-0000000015
images/7c66d2fd-68ee-4628-b860-9d7ad25b2a80.png
pv-test-0000000004
images/84c88143-940d-4b62-bbd2-7acc4cadcb6b.png
pv-train-0000000016
images/c6d97845-8ad2-4ab7-b385-66613628df5b.png
pv-test-0000000005
images/bbf687b5-c6f9-4821-b2e5-a25df1acba47.png
pv-train-0000000017
18 90
pv-

157 450
pv-train-0000000157
158 540
pv-train-0000000158
159 630
pv-train-0000000159
images/3fa1a4e3-c80d-49da-8f7b-a1abb0424f48.png
pv-test-0000000038
images/c6ab535c-30e5-4b6b-8273-1ec8d78cdd13.png
pv-train-0000000160
images/61cd8c7d-394d-42d3-9ed4-e09559110bed.png
pv-test-0000000039
images/755e5bf8-e8e1-492b-a32f-f7d87cccaa40.png
pv-train-0000000161
162 90
pv-train-0000000162
163 180
pv-train-0000000163
164 270
pv-train-0000000164
165 360
pv-train-0000000165
166 450
pv-train-0000000166
167 540
pv-train-0000000167
168 630
pv-train-0000000168
images/2cc3c4ad-670f-41b6-8910-0704cbc192ef.png
pv-test-0000000040
images/b7d0ae11-a393-4aec-8f68-6ba07147f78a.png
pv-train-0000000169
170 90
pv-train-0000000170
171 180
pv-train-0000000171
172 270
pv-train-0000000172
173 360
pv-train-0000000173
174 450
pv-train-0000000174
175 540
pv-train-0000000175
176 630
pv-train-0000000176
images/b7f1a1eb-c150-4456-9c47-e7db8a5cb77a.png
pv-test-0000000041
images/a777e78c-ad1f-4380-9ab2-b4f9f32b2d78.png
pv-tra

images/29dc8176-dfe1-40e0-8247-2dc3c530057e.png
pv-test-0000000070
images/ae17a099-18b5-4a42-a901-850ea056423e.png
pv-train-0000000330
331 90
pv-train-0000000331
332 180
pv-train-0000000332
333 270
pv-train-0000000333
334 360
pv-train-0000000334
335 450
pv-train-0000000335
336 540
pv-train-0000000336
337 630
pv-train-0000000337
images/91827f1a-9b04-4ef0-972c-12e90e60c516.png
pv-test-0000000071
images/29539cb0-1c52-46b2-9302-552f8349dc71.png
pv-train-0000000338
339 90
pv-train-0000000339
340 180
pv-train-0000000340
341 270
pv-train-0000000341
342 360
pv-train-0000000342
343 450
pv-train-0000000343
344 540
pv-train-0000000344
345 630
pv-train-0000000345
images/04628a94-b674-469b-aff6-b476b70df87d.png
pv-test-0000000072
images/ef815e3a-02a6-4cf8-9262-45bc74c50b00.png
pv-train-0000000346
347 90
pv-train-0000000347
348 180
pv-train-0000000348
349 270
pv-train-0000000349
350 360
pv-train-0000000350
351 450
pv-train-0000000351
352 540
pv-train-0000000352
353 630
pv-train-0000000353
images/21c

images/030f3421-ed3f-4bf1-8f27-4d02590bffbc.png
pv-train-0000000501
images/a7f1bbb1-6058-4642-bd66-cfb90dce762e.png
pv-test-0000000103
images/370277f8-436b-4103-8210-b4feb5ab95ea.png
pv-train-0000000502
503 90
pv-train-0000000503
504 180
pv-train-0000000504
505 270
pv-train-0000000505
506 360
pv-train-0000000506
507 450
pv-train-0000000507
508 540
pv-train-0000000508
509 630
pv-train-0000000509
images/c0c3041f-78cd-46de-8e36-0a7dad0c04c4.png
pv-test-0000000104
images/d47cd049-d78b-4ba3-a7d4-399be4dd1dde.png
pv-train-0000000510
511 90
pv-train-0000000511
512 180
pv-train-0000000512
513 270
pv-train-0000000513
514 360
pv-train-0000000514
515 450
pv-train-0000000515
516 540
pv-train-0000000516
517 630
pv-train-0000000517
images/1e86933d-6a5a-4a3c-a0f2-ca22afd767b3.png
pv-test-0000000105
images/0b04ec46-5119-4cda-8c35-c4e5b6f0eed0.png
pv-train-0000000518
519 90
pv-train-0000000519
520 180
pv-train-0000000520
521 270
pv-train-0000000521
522 360
pv-train-0000000522
523 450
pv-train-000000052

679 630
pv-train-0000000679
images/471f75c6-bd5d-4c20-bcc1-258e836f4d69.png
pv-test-0000000133
images/3d302619-473d-4c5b-af43-8833ff94233f.png
pv-train-0000000680
681 90
pv-train-0000000681
682 180
pv-train-0000000682
683 270
pv-train-0000000683
684 360
pv-train-0000000684
685 450
pv-train-0000000685
686 540
pv-train-0000000686
687 630
pv-train-0000000687
images/04018590-4e66-4cb9-ad2d-83570f1b29ae.png
pv-test-0000000134
images/6f4ffe7b-395f-41ea-ad61-df5e0cbd531c.png
pv-train-0000000688
689 90
pv-train-0000000689
690 180
pv-train-0000000690
691 270
pv-train-0000000691
692 360
pv-train-0000000692
693 450
pv-train-0000000693
694 540
pv-train-0000000694
695 630
pv-train-0000000695
images/f329e01b-fbd3-4fb5-9738-a7ff4c8622b8.png
pv-test-0000000135
images/56077789-8580-41b4-89d6-deea227e5c60.png
pv-train-0000000696
697 90
pv-train-0000000697
698 180
pv-train-0000000698
699 270
pv-train-0000000699
700 360
pv-train-0000000700
701 450
pv-train-0000000701
702 540
pv-train-0000000702
703 630
pv

...

images/997eabd9-55bf-4683-83c4-4cee4673a4ef.png
pv-train-0000001836
1837 90
pv-train-0000001837
1838 180
pv-train-0000001838
1839 270
pv-train-0000001839
1840 360
pv-train-0000001840
1841 450
pv-train-0000001841
1842 540
pv-train-0000001842
1843 630
pv-train-0000001843
images/820bbfea-3dae-46fc-b61a-3d0ed2ea01c2.png
pv-test-0000000366
images/3d1b889e-bb4a-4ee6-b899-974ee7d956a7.png
pv-train-0000001844
images/58ced55c-bf0d-4204-b6b2-4f8992d92779.png
pv-test-0000000367
images/1dd3e6dd-343f-4cc9-b0ed-e4d27fb36f0a.png
pv-train-0000001845
1846 90
pv-train-0000001846
1847 180
pv-train-0000001847
1848 270
pv-train-0000001848
1849 360
pv-train-0000001849
1850 450
pv-train-0000001850
1851 540
pv-train-0000001851
1852 630
pv-train-0000001852
images/477e541c-0519-4542-9b13-8ee7922644d3.png
pv-test-0000000368
images/3e753a44-3264-438c-8389-a7822e4a84cc.png
pv-train-0000001853
1854 90
pv-train-0000001854
1855 180
pv-train-0000001855
1856 270
pv-train-0000001856
1857 360
pv-train-0000001857
1858 450

2008 90
pv-train-0000002008
2009 180
pv-train-0000002009
2010 270
pv-train-0000002010
2011 360
pv-train-0000002011
2012 450
pv-train-0000002012
2013 540
pv-train-0000002013
2014 630
pv-train-0000002014
images/216f9d43-6869-4ba1-9106-4f5ea6aac223.png
pv-test-0000000397
images/b4f3b2a4-f4b9-48de-ac5d-0e681b77be27.png
pv-train-0000002015
2016 90
pv-train-0000002016
2017 180
pv-train-0000002017
2018 270
pv-train-0000002018
2019 360
pv-train-0000002019
2020 450
pv-train-0000002020
2021 540
pv-train-0000002021
2022 630
pv-train-0000002022
images/baa87bf8-786c-444c-81e8-9fa0a41d8149.png
pv-test-0000000398
images/6afb3717-f197-4668-929a-4fab389ae6e8.png
pv-train-0000002023
images/d0347b49-8f98-4632-a384-1a4f57ac6374.png
pv-test-0000000399
images/592c941a-aa54-4971-b499-dc47356f80a3.png
pv-train-0000002024
2025 90
pv-train-0000002025
2026 180
pv-train-0000002026
2027 270
pv-train-0000002027
2028 360
pv-train-0000002028
2029 450
pv-train-0000002029
2030 540
pv-train-0000002030
2031 630
pv-train-

2187 450
pv-train-0000002187
2188 540
pv-train-0000002188
2189 630
pv-train-0000002189
images/4fb83278-adc5-4424-9f45-b6d0db2955b6.png
pv-test-0000000426
images/d67655e6-0a80-4a86-b552-1f5e85788377.png
pv-train-0000002190
2191 90
pv-train-0000002191
2192 180
pv-train-0000002192
2193 270
pv-train-0000002193
2194 360
pv-train-0000002194
2195 450
pv-train-0000002195
2196 540
pv-train-0000002196
2197 630
pv-train-0000002197
images/43072c64-4c08-47cd-ae8a-b74d31f80008.png
pv-test-0000000427
images/a27b451d-44a6-4740-861a-9e7e99915563.png
pv-train-0000002198
2199 90
pv-train-0000002199
2200 180
pv-train-0000002200
2201 270
pv-train-0000002201
2202 360
pv-train-0000002202
2203 450
pv-train-0000002203
2204 540
pv-train-0000002204
2205 630
pv-train-0000002205
images/c32b8bb8-c89d-4ae9-9c77-5b9356ca35a1.png
pv-test-0000000428
images/68f34a9c-f73d-4238-8f55-4b05a8a2154c.png
pv-train-0000002206
2207 90
pv-train-0000002207
2208 180
pv-train-0000002208
2209 270
pv-train-0000002209
2210 360
pv-train-

2372 450
pv-train-0000002372
2373 540
pv-train-0000002373
2374 630
pv-train-0000002374
images/e4827c85-311a-4df1-89c2-99dad0c3558c.png
pv-test-0000000453
images/434523fd-d4d9-4961-a7c2-d71d6ae68e13.png
pv-train-0000002375
2376 90
pv-train-0000002376
2377 180
pv-train-0000002377
2378 270
pv-train-0000002378
3191 450
pv-train-0000003191
3192 540
pv-train-0000003192
3193 630
pv-train-0000003193
images/50d7c6c9-3230-48fe-947c-7dcfd0aad710.png
pv-test-0000000572
images/d12c5f45-6b20-46d6-a66f-c5a4683c25b3.png
pv-train-0000003194
3195 90
pv-train-0000003195
3196 180
pv-train-0000003196
3197 270
pv-train-0000003197
3198 360
pv-train-0000003198
3199 450
pv-train-0000003199
3200 540
pv-train-0000003200
3201 630
pv-train-0000003201
images/f48bf825-13c4-42fc-8dd8-10cc4ae6b202.png
pv-test-0000000573
images/c4c7ed75-dbdf-4bb6-b6ba-718903cf7264.png
pv-train-0000003202
3203 90
pv-train-0000003203
3204 180
pv-train-0000003204
3205 270
pv-train-0000003205
3206 360
pv-train-0000003206
3207 450
pv-train-

3367 90
pv-train-0000003367
3368 180
pv-train-0000003368
3369 270
pv-train-0000003369
3370 360
pv-train-0000003370
3371 450
pv-train-0000003371
3372 540
pv-train-0000003372
3373 630
pv-train-0000003373
images/05ac7b2a-6a77-4464-926b-8570f7c01693.png
pv-test-0000000599
images/dcb21739-3c36-459d-b3cc-1be5871d6fe7.png
pv-train-0000003374
3375 90
pv-train-0000003375
3376 270
pv-train-0000003376
3377 360
pv-train-0000003377
3378 450
pv-train-0000003378
3379 540
pv-train-0000003379
3380 630
pv-train-0000003380
images/a2fc0093-8645-43c7-bdee-ec8ca9484815.png
pv-test-0000000600
images/161978c3-3a05-47a5-9fee-be7d25693eae.png
pv-train-0000003381
3382 90
pv-train-0000003382
3383 180
pv-train-0000003383
3384 270
pv-train-0000003384
3385 360
pv-train-0000003385
3386 450
pv-train-0000003386
3387 540
pv-train-0000003387
3388 630
pv-train-0000003388
images/a4409217-be11-4478-90a5-b406057d154d.png
pv-test-0000000601
images/03b9d6ad-39ca-4097-a98b-77f02b3c00a7.png
pv-train-0000003389
3390 90
pv-train-0

In [4]:
print(phase_count["train"])
print(phase_count["test"])

3419
605


In [5]:
# Training and Validate Sets
# degs = [45,90,135,180,225,270,315]
img_bbs = []
# phase_count["train"] = 3484
labels = os.path.join('test.json')
with open(labels,'r') as json_file:
    datas = json.load(json_file)
    for data in datas:
        read_path = (data['image']['pathname'])[1:]
        print(read_path)
        phase = "train"
        img_name_new = "pv-" + phase + "-{:010d}".format(phase_count[phase])
        print(img_name_new)
        save_path = os.path.join('../data/custom/images',img_name_new+".jpg")
        img = Image.open(read_path)
        objects = data['objects']
        rects = []
        result = ""
        img = np.array(img)
        h,w,c = img.shape
        for obj in objects:
            minr = obj['bounding_box']['minimum']['r']
            minc = obj['bounding_box']['minimum']['c']
            maxr = obj['bounding_box']['maximum']['r']
            maxc = obj['bounding_box']['maximum']['c']
            x1 = minc / w
            y1 = minr / h
            bw = (maxc - minc) / w
            bh = (maxr - minr) / h
            xc = x1 + (bw / 2)
            yc = y1 + (bh / 2)
            category = obj['category']
            if (category == "red blood cell"):
                result += "\n"+ "0 " + str(xc) + " " + str(yc) + " " + str(bw) + " " + str(bh)
            elif category == "ring" or category == "trophozoite" or category == "schizont" or category == "gametocyte":
                result += "\n" + "1 " + str(xc) + " " + str(yc) + " " + str(bw) + " " + str(bh)
                rects.append(BoundingBox(x1=minc,y1=minr,x2=maxc,y2=maxr,label=1))
        result = result[1:]
        Image.fromarray(img).save(save_path)
        f = open(os.path.join('../data/custom/labels',img_name_new+".txt"),'w')
        f.write(result)
        f.close()
        bbs = BoundingBoxesOnImage(rects,shape=img.shape)
        count += 1
        phase_count[phase] += 1
        if phase == "train":
            for deg in degs:
                image_aug, bbs_aug = iaa.Affine(rotate=deg,translate_percent={"x": (-0.25, 0.25), "y": (-0.25, 0.25)},mode="reflect")(image=img, bounding_boxes=bbs)
                clip_off_bbs = bbs_aug.remove_out_of_image(fully=True,partly=True).clip_out_of_image()
                h,w,c = image_aug.shape
                result = ""
                img_name_new = "pv-" + phase + "-{:010d}".format(phase_count[phase])
                
                for bb in clip_off_bbs.bounding_boxes:
                    minx = int(bb.x1)
                    miny = int(bb.y1)
                    maxx = int(bb.x2)
                    maxy = int(bb.y2)
                    bb_w = (maxx-minx)/w
                    bb_h = (maxy-miny)/h
                    bb_xc = (minx + ((maxx-minx)/2))/w
                    bb_yc = (miny + ((maxy-miny)/2))/h
                    result += "\n" + str(bb.label)+" "+str(bb_xc)+" "+str(bb_yc)+ " "+str(bb_w)+ " "+str(bb_h)
                if len(clip_off_bbs.bounding_boxes) > 0:
                    print("{} {}".format(phase_count[phase],deg))
                    print(img_name_new)
                    result = result[1:]
                    save_path = os.path.join('../data/custom/images',img_name_new+".jpg")
                    Image.fromarray(image_aug).save(save_path)
                    f = open(os.path.join('../data/custom/labels',img_name_new+".txt"),'w')
                    f.write(result)
                    f.close()
                    phase_count[phase] += 1

images/41be1bd3-0d31-4881-bf1f-3ccdfa21ff12.jpg
pv-train-0000003419
3420 90
pv-train-0000003420
3421 180
pv-train-0000003421
3422 270
pv-train-0000003422
3423 360
pv-train-0000003423
3424 450
pv-train-0000003424
3425 540
pv-train-0000003425
3426 630
pv-train-0000003426
images/64985a1e-16bb-4016-a01c-c21a5b86e572.jpg
pv-train-0000003427
3428 90
pv-train-0000003428
3429 180
pv-train-0000003429
3430 270
pv-train-0000003430
3431 360
pv-train-0000003431
3432 450
pv-train-0000003432
3433 540
pv-train-0000003433
3434 630
pv-train-0000003434
images/c70894fa-50e8-4eed-a5a8-159a20153a49.jpg
pv-train-0000003435
3436 180
pv-train-0000003436
3437 270
pv-train-0000003437
3438 360
pv-train-0000003438
3439 450
pv-train-0000003439
3440 540
pv-train-0000003440
3441 630
pv-train-0000003441
images/abb72ce2-4193-4dd3-a34c-61c2ee22ccff.jpg
pv-train-0000003442
3443 90
pv-train-0000003443
3444 180
pv-train-0000003444
3445 270
pv-train-0000003445
3446 360
pv-train-0000003446
3447 450
pv-train-0000003447
3448 5

3660 630
pv-train-0000003660
images/e6a7743e-4040-40fa-b097-ca6874c469b7.jpg
pv-train-0000003661
3662 90
pv-train-0000003662
3663 180
pv-train-0000003663
3664 270
pv-train-0000003664
3665 360
pv-train-0000003665
3666 450
pv-train-0000003666
3667 540
pv-train-0000003667
3668 630
pv-train-0000003668
images/e02301ac-68f7-4daf-b2ba-254b917f1cca.jpg
pv-train-0000003669
3670 90
pv-train-0000003670
3671 180
pv-train-0000003671
3672 270
pv-train-0000003672
3673 360
pv-train-0000003673
3674 450
pv-train-0000003674
3675 540
pv-train-0000003675
3676 630
pv-train-0000003676
images/1e647046-333b-4102-a0c6-4c29405c37b3.jpg
pv-train-0000003677
3678 90
pv-train-0000003678
3679 180
pv-train-0000003679
3680 270
pv-train-0000003680
3681 360
pv-train-0000003681
3682 450
pv-train-0000003682
3683 540
pv-train-0000003683
3684 630
pv-train-0000003684
images/ef55f2ac-5a04-4986-99ab-0d2b5052ab3e.jpg
pv-train-0000003685
3686 90
pv-train-0000003686
3687 180
pv-train-0000003687
3688 270
pv-train-0000003688
3689 36

3902 450
pv-train-0000003902
3903 540
pv-train-0000003903
3904 630
pv-train-0000003904
images/224c80b8-6c55-48cf-b719-96a47d223dc3.jpg
pv-train-0000003905
3906 90
pv-train-0000003906
3907 180
pv-train-0000003907
3908 270
pv-train-0000003908
3909 360
pv-train-0000003909
3910 450
pv-train-0000003910
3911 540
pv-train-0000003911
3912 630
pv-train-0000003912
images/51a42643-256f-463a-bf22-7fd295caead0.jpg
pv-train-0000003913
3914 90
pv-train-0000003914
3915 180
pv-train-0000003915
3916 270
pv-train-0000003916
3917 360
pv-train-0000003917
3918 450
pv-train-0000003918
3919 540
pv-train-0000003919
3920 630
pv-train-0000003920
images/f8ebe0c8-a90d-4a64-9f6a-f9985e38c340.jpg
pv-train-0000003921
3922 90
pv-train-0000003922
3923 180
pv-train-0000003923
3924 270
pv-train-0000003924
3925 360
pv-train-0000003925
3926 450
pv-train-0000003926
3927 540
pv-train-0000003927
3928 630
pv-train-0000003928
images/4b3de416-ac61-43eb-80d0-36b81c4baf82.jpg
pv-train-0000003929
3930 90
pv-train-0000003930
3931 18

4144 270
pv-train-0000004144
4145 360
pv-train-0000004145
4146 450
pv-train-0000004146
4147 540
pv-train-0000004147
4148 630
pv-train-0000004148
images/b1d312cf-1f52-4955-bf42-58d3664c254a.jpg
pv-train-0000004149
4150 90
pv-train-0000004150
4151 180
pv-train-0000004151
4152 270
pv-train-0000004152
4153 360
pv-train-0000004153
4154 450
pv-train-0000004154
4155 630
pv-train-0000004155
images/5c399ab9-492c-4bfc-a452-83aba574a3bc.jpg
pv-train-0000004156
4157 90
pv-train-0000004157
4158 180
pv-train-0000004158
4159 270
pv-train-0000004159
4160 360
pv-train-0000004160
4161 450
pv-train-0000004161
4162 540
pv-train-0000004162
4163 630
pv-train-0000004163
images/96957ec3-de17-41cf-855b-1388814e5918.jpg
pv-train-0000004164
4165 90
pv-train-0000004165
4166 180
pv-train-0000004166
4167 270
pv-train-0000004167
4168 360
pv-train-0000004168
4169 450
pv-train-0000004169
images/291fe50c-2134-4708-bfa4-09b36ffea123.jpg
pv-train-0000004170
4171 90
pv-train-0000004171
4172 180
pv-train-0000004172
4173 27

In [6]:
print(phase_count["train"])
print(phase_count["test"])

4316
605


In [7]:
# get images from pv_test (Only test)
labels = os.path.join('pv_test.json')

# phase_count["test"] = 605

phase = "test"
# degs = [45,90,135,180,225,270,315]

with open(labels,'r') as json_file:
    data = json.load(json_file)
    images = data["_via_img_metadata"]
    for key in images:
        url = images[key]["filename"]
        leng = len(images[key]["regions"])
        bboxs = ""
        if leng > 0:
            name = "pv-" + phase + "-{:010d}".format(phase_count[phase])
            print("{} {}".format(count,url))
            print(name)
            response = requests.get(url)
            img = Image.open(BytesIO(response.content))
            w, h = img.size
            img.save(os.path.join('../data/custom/images',name+".jpg"))
            f = open(os.path.join('../data/custom/labels',name+".txt"),'w')
            rects = []
            for box in images[key]["regions"]:
                b = box["shape_attributes"]
                x1 = b["x"]/w
                y1 = b["y"]/h
                bw = b["width"]/w
                bh = b["height"]/h
                xc = x1 + (bw/2)
                yc = y1 + (bh/2)
                lbl = box["region_attributes"]["type"]
                if lbl == "1":
                    lbl = "1"
                    bboxs += "\n"+ lbl + " " + str(xc) + " " + str(yc) + " " + str(bw) + " " + str(bh)
                    rects.append(BoundingBox(x1=b["x"],y1=b["y"],x2=(b["x"] + b["width"]),y2=(b["y"] + b["height"]),label=lbl))
                elif lbl == "0":
                    lbl = "0"
                    bboxs += "\n"+ lbl + " " + str(xc) + " " + str(yc) + " " + str(bw) + " " + str(bh)
            bboxs = bboxs[1:]
            f.write(bboxs)
            f.close()
            img = np.array(img)
            bbs = BoundingBoxesOnImage(rects,shape=img.shape)
            count += 1
            phase_count[phase] += 1

1329 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv_test/1.jpg
pv-test-0000000605
1330 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv_test/10.jpg
pv-test-0000000606
1331 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv_test/13.jpg
pv-test-0000000607
1332 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv_test/17.jpg
pv-test-0000000608
1333 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv_test/25.jpg
pv-test-0000000609
1334 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv_test/26.jpg
pv-test-0000000610
1335 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv_test/30.jpg
pv-test-0000000611
1336 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv_test/32.jpg
pv-test-0000000612
1337 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv_test/33.jpg
pv-test-0000000613
1338 https://raw.githubuserco

In [8]:
print(phase_count["train"])
print(phase_count["test"])

4316
633


In [9]:
# labels = os.path.join('pv_ext.json')
phase = "train"

data = (requests.get('https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv_ext.json')).json()
images = data["_via_img_metadata"]
for key in images:
    url = images[key]["filename"]
    leng = len(images[key]["regions"])
    bboxs = ""
    if leng > 0:
        name = "pv-" + phase + "-{:010d}".format(phase_count[phase])
        print("{} {}".format(phase_count[phase],url))
        print(name)
        response = requests.get(url)
        img = Image.open(BytesIO(response.content))
        w, h = img.size
        img.save(os.path.join('../data/custom/images',name+".jpg"))
        f = open(os.path.join('../data/custom/labels',name+".txt"),'w')
        rects = []
        for box in images[key]["regions"]:
            b = box["shape_attributes"]
            x1 = b["x"]/w
            y1 = b["y"]/h
            bw = b["width"]/w
            bh = b["height"]/h
            xc = x1 + (bw/2)
            yc = y1 + (bh/2)
            lbl = box["region_attributes"]["type"]
            if lbl == "1":
                lbl = "1"
                bboxs += "\n"+ lbl + " " + str(xc) + " " + str(yc) + " " + str(bw) + " " + str(bh)
                rects.append(BoundingBox(x1=b["x"],y1=b["y"],x2=(b["x"] + b["width"]),y2=(b["y"] + b["height"]),label=lbl))
            elif lbl == "0":
                lbl = "0"
                bboxs += "\n"+ lbl + " " + str(xc) + " " + str(yc) + " " + str(bw) + " " + str(bh)
        bboxs = bboxs[1:]
        f.write(bboxs)
        f.close()
        img = np.array(img)
        bbs = BoundingBoxesOnImage(rects,shape=img.shape)
        phase_count[phase] += 1
        for deg in degs:
            name = "pv-" + phase + "-{:010d}".format(phase_count[phase])
            image_aug, bbs_aug = iaa.Affine(rotate=deg,translate_percent={"x": (-0.25, 0.25), "y": (-0.25, 0.25)},mode="reflect")(image=img, bounding_boxes=bbs)
            aug_h, aug_w, aug_c = image_aug.shape
            clip_off_bbs = bbs_aug.remove_out_of_image(fully=True,partly=True).clip_out_of_image()
            bboxs = ""
            for bb in clip_off_bbs.bounding_boxes:
                x1 = bb.x1
                y1 = bb.y1
                x2 = bb.x2
                y2 = bb.y2
                bb_w = (x2-x1)/aug_w
                bb_h = (y2-y1)/aug_h
                bb_xc = (x1/aug_w) + (bb_w/2)
                bb_yc = (y1/aug_h) + (bb_h/2)
                bboxs += "\n"+ str(bb.label) + " " + str(bb_xc) + " " + str(bb_yc) + " " + str(bb_w) + " " + str(bb_h)
            if len(clip_off_bbs.bounding_boxes) > 0:
                bboxs = bboxs[1:]
                im_aug = Image.fromarray(image_aug)
                im_aug.save(os.path.join('../data/custom/images',name+".jpg"))
                f = open(os.path.join('../data/custom/labels',name+".txt"),'w')
                f.write(bboxs)
                f.close()
                print("{} {}".format(phase_count[phase],deg))
                phase_count[phase] += 1

4316 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000001.jpg
pv-train-0000004316
4317 90
4318 180
4319 270
4320 360
4321 450
4322 540
4323 630
4324 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000002.jpg
pv-train-0000004324
4325 90
4326 180
4327 270
4328 360
4329 450
4330 540
4331 630
4332 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000003.jpg
pv-train-0000004332
4333 90
4334 180
4335 270
4336 360
4337 450
4338 540
4339 630
4340 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000004.jpg
pv-train-0000004340
4341 90
4342 180
4343 270
4344 450
4345 540
4346 630
4347 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000006.jpg
pv-train-0000004347
4348 90
4349 180
4350 270
4351 360
4352 450
4353 540
4354 630
4355 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000007.jpg
pv-train-0000004355
4356 90
4357 180
4358 270
4

4680 180
4681 360
4682 540
4683 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000051.jpg
pv-train-0000004683
4684 90
4685 180
4686 270
4687 360
4688 450
4689 540
4690 630
4691 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000052.jpg
pv-train-0000004691
4692 180
4693 360
4694 450
4695 540
4696 630
4697 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000053.jpg
pv-train-0000004697
4698 90
4699 270
4700 360
4701 450
4702 540
4703 630
4704 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000054.jpg
pv-train-0000004704
4705 90
4706 180
4707 270
4708 360
4709 450
4710 540
4711 630
4712 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000055.jpg
pv-train-0000004712
4713 90
4714 180
4715 270
4716 360
4717 450
4718 540
4719 630
4720 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000056.jpg
pv-train-0000004720
4721 90
4722 180


4999 630
5000 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000103.jpg
pv-train-0000005000
5001 180
5002 360
5003 450
5004 540
5005 630
5006 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000104.jpg
pv-train-0000005006
5007 90
5008 180
5009 270
5010 360
5011 540
5012 630
5013 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000105.jpg
pv-train-0000005013
5014 90
5015 180
5016 270
5017 360
5018 450
5019 630
5020 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000106.jpg
pv-train-0000005020
5021 180
5022 270
5023 360
5024 450
5025 540
5026 630
5027 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000107.jpg
pv-train-0000005027
5028 90
5029 180
5030 270
5031 360
5032 540
5033 630
5034 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000108.jpg
pv-train-0000005034
5035 90
5036 180
5037 360
5038 450
5039 https://raw.githubuse

5319 90
5320 180
5321 270
5322 450
5323 540
5324 630
5325 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000155.jpg
pv-train-0000005325
5326 180
5327 270
5328 360
5329 540
5330 630
5331 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000156.jpg
pv-train-0000005331
5332 90
5333 180
5334 270
5335 360
5336 450
5337 540
5338 630
5339 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000157.jpg
pv-train-0000005339
5340 90
5341 180
5342 360
5343 450
5344 540
5345 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000158.jpg
pv-train-0000005345
5346 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000159.jpg
pv-train-0000005346
5347 90
5348 270
5349 360
5350 450
5351 540
5352 630
5353 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000160.jpg
pv-train-0000005353
5354 90
5355 180
5356 270
5357 360
5358 450
5359 540
5360 630
5361 http

5630 270
5631 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000208.jpg
pv-train-0000005631
5632 270
5633 450
5634 540
5635 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000209.jpg
pv-train-0000005635
5636 90
5637 180
5638 270
5639 450
5640 540
5641 630
5642 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000210.jpg
pv-train-0000005642
5643 90
5644 180
5645 270
5646 360
5647 450
5648 540
5649 630
5650 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000211.jpg
pv-train-0000005650
5651 90
5652 180
5653 270
5654 360
5655 450
5656 540
5657 630
5658 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000212.jpg
pv-train-0000005658
5659 180
5660 270
5661 360
5662 450
5663 540
5664 630
5665 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000213.jpg
pv-train-0000005665
5666 90
5667 180
5668 270
5669 450
5670 540
5671 https://raw.

...

7856 630
7857 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000575.jpg
pv-train-0000007857
7858 90
7859 180
7860 270
7861 450
7862 540
7863 630
7864 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000576.jpg
pv-train-0000007864
7865 540
7866 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000577.jpg
pv-train-0000007866
7867 90
7868 180
7869 270
7870 360
7871 450
7872 540
7873 630
7874 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000578.jpg
pv-train-0000007874
7875 90
7876 180
7877 270
7878 360
7879 450
7880 540
7881 630
7882 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000579.jpg
pv-train-0000007882
7883 90
7884 180
7885 270
7886 360
7887 450
7888 630
7889 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000580.jpg
pv-train-0000007889
7890 90
7891 360
7892 450
7893 540
7894 630
7895 https://raw.githubusercontent.c

8201 90
8202 180
8203 270
8204 450
8205 540
8206 630
8207 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000626.jpg
pv-train-0000008207
8208 90
8209 180
8210 270
8211 360
8212 450
8213 540
8214 630
8215 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000627.jpg
pv-train-0000008215
8216 90
8217 180
8218 270
8219 360
8220 450
8221 540
8222 630
8223 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000628.jpg
pv-train-0000008223
8224 360
8225 540
8226 630
8227 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000629.jpg
pv-train-0000008227
8228 90
8229 270
8230 450
8231 540
8232 630
8233 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000630.jpg
pv-train-0000008233
8234 90
8235 180
8236 360
8237 450
8238 540
8239 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000631.jpg
pv-train-0000008239
8240 360
8241 450
8242 540
8243 http

8532 90
8533 180
8534 270
8535 360
8536 450
8537 540
8538 630
8539 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000677.jpg
pv-train-0000008539
8540 90
8541 180
8542 270
8543 360
8544 450
8545 540
8546 630
8547 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000678.jpg
pv-train-0000008547
8548 90
8549 180
8550 270
8551 360
8552 450
8553 540
8554 630
8555 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000679.jpg
pv-train-0000008555
8556 90
8557 180
8558 270
8559 360
8560 450
8561 540
8562 630
8563 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000680.jpg
pv-train-0000008563
8564 90
8565 360
8566 540
8567 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000681.jpg
pv-train-0000008567
8568 90
8569 180
8570 270
8571 360
8572 450
8573 540
8574 630
8575 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000682.jpg
pv-train-000

8875 90
8876 180
8877 270
8878 360
8879 630
8880 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000728.jpg
pv-train-0000008880
8881 90
8882 180
8883 270
8884 360
8885 450
8886 540
8887 630
8888 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000729.jpg
pv-train-0000008888
8889 90
8890 180
8891 270
8892 450
8893 540
8894 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000730.jpg
pv-train-0000008894
8895 90
8896 270
8897 360
8898 450
8899 540
8900 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000731.jpg
pv-train-0000008900
8901 90
8902 180
8903 270
8904 360
8905 630
8906 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000732.jpg
pv-train-0000008906
8907 90
8908 180
8909 270
8910 540
8911 630
8912 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000733.jpg
pv-train-0000008912
8913 90
8914 180
8915 270
8916 360
8917 450
89

9195 540
9196 630
9197 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000780.jpg
pv-train-0000009197
9198 90
9199 180
9200 270
9201 360
9202 540
9203 630
9204 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000781.jpg
pv-train-0000009204
9205 90
9206 180
9207 270
9208 360
9209 450
9210 540
9211 630
9212 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000782.jpg
pv-train-0000009212
9213 90
9214 180
9215 270
9216 360
9217 450
9218 540
9219 630
9220 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000783.jpg
pv-train-0000009220
9221 180
9222 360
9223 540
9224 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000784.jpg
pv-train-0000009224
9225 180
9226 270
9227 360
9228 630
9229 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000785.jpg
pv-train-0000009229
9230 90
9231 180
9232 270
9233 360
9234 450
9235 540
9236 630
9237 htt

In [10]:
print(phase_count["train"])
print(phase_count["test"])

9486
633


In [11]:
# # summary test set
# files = glob.glob("images/pv-test-*.jpg")
# count = 0
# count_class = [0,0]

# for file in files:
#     img = file.split("/")[-1]
#     name = img.split('.')[0]
#     f = open("labels/"+name+".txt","r")
#     lines = list(f)
#     f.close()
#     print(img)
#     for line in lines:
# #         print(line)
#         class_id, xc, yc, wb, hb = line.split()
#         count_class[int(class_id)] += 1
#     count += 1
# print(count)
# print(count_class)

In [12]:
# print(count)
# print(count_class)

In [13]:
# # summary train set
# files = glob.glob("images/pv-train-*.jpg")
# count = 0
# count_class = [0,0]

# for file in files:
#     img = file.split("/")[-1]
#     name = img.split('.')[0]
#     f = open("labels/"+name+".txt","r")
#     lines = list(f)
#     f.close()
#     print(img)
#     for line in lines:
# #         print(line)
#         class_id, xc, yc, wb, hb = line.split()
#         count_class[int(class_id)] += 1
#     count += 1
# print(count)
# print(count_class)

In [14]:
# print(count)
# print(count_class)

In [15]:
# clear empty box
# files = glob.glob("images/pv-train-*.jpg")
# count = 0
# for file in files:
#     img = file.split("/")[-1]
#     name = img.split('.')[0]
#     f = open("labels/"+name+".txt","r")
#     txt = list(f)
#     f.close()
#     if len(txt) == 0:
#         print(img)
#         os.remove("labels/"+name+".txt")
#         os.remove(file)
#         count += 1
# print(count)

In [16]:
# img = imread('tmp_images/images/2559636b-f01a-4414-93da-210c3b12d153.png')
# image_aug = iaa.Affine(rotate=90,translate_percent={"x": (-0.25, 0.25), "y": (-0.25, 0.25)},mode="reflect")(image=img)
# imsave('aug02.png',image_aug)